In [ ]:
%matplotlib inline
import numpy as np
#from sklearn.gaussian_process import GaussianProcessRegressor
#from sklearn import preprocessing
import george
from george import gp
from astropy.io import fits as pyfits
import matplotlib.pyplot as plt


# Investigating spectral inter/extrapolation on synthetic stellar spectra with Gaussian Processes

In [ ]:
def get_training_data(datadir='/Users/prunet/Dropbox/Work/Gaia/', scale_data=True):
    
    # Get parameter values Teff, log(g), [M/H], [alpha/Fe]
    Xtrain = np.loadtxt(datadir+'RVS2020_Grid4.Param',usecols=(1,2,3,4))
    n_samples_train = Xtrain.shape[0]
    
    # Get corresponding spectra, check that dimensions are compatible
    Ytrain = pyfits.getdata(datadir+'RVS2020_Grid4_LR.fits').T
    n_samples_spec = Ytrain.shape[0]
    
    if (n_samples_train != n_samples_spec):
        print('Incompatible dimensions between parameters list size (%d) and number of spectra (%d)'
              %(n_samples_train,n_samples_spec))
    if not scale_data:
        return Xtrain, Ytrain
    else:
        scaler = preprocessing.RobustScaler().fit(Xtrain)
        Xtrain_scaled = scaler.transform(Xtrain)
        return Xtrain_scaled, Ytrain, scaler


## Get data, scale input features, compute scalar training toy model (norm of spectra, for a start)

In [ ]:
Xtrain,Ytrain,scaler = get_training_data(scale_data=True)
ytrain_power = np.linalg.norm(Ytrain,axis=1)

## Now create GaussianProcessRegressor instance. Train it on data. Use RBF kernel, hyperparameters (amplitude, length scale) will be adjusted during training by minimizing the marginal likelihood

In [ ]:
## USE HOLDR fast solver of gp here... + explicit marginal likelihood maximisation

kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 10.0))
gpr = GaussianProcessRegressor(kernel=kernel, random_state=0)
gpr.fit(Xtrain,ytrain_power)


In [ ]:
plt.scatter(Xtrain[:,2],Xtrain[:,3],c=np.linalg.norm(Ytrain,axis=1))